# Testing the BYM2 Model: The Penalised-Complexity Priors  

### Goals of this session  
1. Simulate data with different underlying random effects components.  
    - $\delta$ which is pure overdispersion  
    - $S$ which is spatially correlated  
    - A combination of $\delta$ and $S$  
2. Use the BYM2 model to asses how well the model does at caputuring these effects.  
3. simulate a similar set of data this time using covarites.  
4. Asses covariate bias in BYM2, overdispersion, GMRF, and GMRF+Overdispersion model.  
5. Use the ohio dataset as real test case to models  

In [3]:
set.seed(123)
rm(list=ls())
library(SpatialEpi)
library(spdep)
library(INLA)
library(RColorBrewer)
library(maptools)
library(maps)
library(ggplot2)
library(sp)
library(lattice)
library(surveillance)
library(parallel)

## Get Pennsylvania map file
setwd("~/Documents/Classes/spatial_epi/project/")
load("../homework/hw3/USA_adm2.RData") # From http://gadm.org/

# define the 50 states remove DC
states <- unique(gadm$NAME_1)
states <- as.character(states[!(states %in% c("District of Columbia"))])

# each data unit will be a state with different data
state_data <- lapply(states, function(x) list(name=x))
names(state_data) <- states

# while testing lets only keep three states
state_data <- state_data[c("California", "Washington")]

apply2 <- function(f, new_key, state_data, cores=1){
    # apply a function in parallel to an old key of state data and then
    # add the newely generated data to the key list
    states <- names(state_data)
    temp <- mclapply(states, function(x) 
        f(state_data[[x]]), mc.cores=cores)
    names(temp) <- states
    for(state in states){
        state_data[[state]][[new_key]] <- temp[[state]]
    }
    state_data
}

state_data <- apply2(function(x) as(gadm[which(gadm$NAME_1==x$name),], "SpatialPolygons"), 
                     "SpatialPolygons", state_data, cores=3)

state_data <- apply2(function(x) poly2adjmat(x$SpatialPolygons), 
                     "adjmat", state_data, cores=3)

distance_matrix <- function(tmat){
    loops <- 0
    islands <- names(colSums(tmat)[colSums(tmat) == 0])
    sans_island <- tmat
    sans_island[,] <- TRUE
    sans_island[islands,] <- FALSE
    sans_island[,islands] <- FALSE
    while(sum(tmat[lower.tri(tmat, diag = FALSE) & sans_island] == 0) != 0){
        loops <- loops + 1
        for(i in 1:(nrow(tmat) - 1)){
            for(j in (i+1):(nrow(tmat))){
                if(tmat[i,j] == 0){
                    i_neighbors <- tmat[i,][tmat[i,] != 0]
                    j_neighbors <- tmat[j,][tmat[j,] != 0]
                    shared <- intersect(names(i_neighbors), names(j_neighbors))
                    min_ <- nrow(tmat)
                    for(s in shared){
                        distance <- j_neighbors[s] + i_neighbors[s]
                        if(distance < min_){
                            min_ <- distance
                        }
                    }
                    if(length(s) > 0 & min_ <= (loops + 1)){
                        tmat[i,j] <- min_
                        tmat[j,i] <- min_
                    }
                }
            }
        }
    }
    tmat[islands,] <- nrow(tmat) * 3
    tmat[,islands] <- nrow(tmat) * 3
    tmat[diag(tmat)] <- 0
    return (tmat)
}

rmvn <- function(n, mu = 0, V = matrix(1)) {
    # http://rstudio-pubs-static.s3.amazonaws.com/9688_a49c681fab974bbca889e3eae9fbb837.html
    p <- length(mu)
    if (any(is.na(match(dim(V), p)))) 
        stop("Dimension problem!")
    D <- chol(V)
    t(matrix(rnorm(n * p), ncol = p) %*% D + rep(mu, rep(n, p)))
}

simulate_sre <- function(dist_mat){
    N <- nrow(dist_mat)
    phi <- 0.38
    rmvn(1, rep(0, N), exp(-phi * dist_mat))
} 

delta_simulatar <- function(unit){
    # Given a spatial polygons data set applies
    # a delta to each unit
    sp <- unit$SpatialPolygons
    dist_mat <- distance_matrix(unit$adjmat)
    N <- length(sp) # number of counties in a state unit
    sp$ID <- 1:N
    
    datur_od <- sapply(1:100, function(x) rnorm(length(sp), 0, 1))
    datur_se <- sapply(1:100, function(x) simulate_sre(dist_mat))
    x1 <- sapply(1:100, function(x)rnorm(length(sp)))
    x2 <- sapply(1:100, function(x)rnorm(length(sp)))
    beta1 <- 2
    beta2 <- -3
    mean_ <- beta1 * x1 + beta2 * x2
    random <- sapply(1:ncol(datur_od), function(x) 
        rpois(length(datur_od[,x]), exp(datur_od[,x])))
    spatial <- sapply(1:ncol(datur_se), function(x) 
        rpois(length(datur_se[,x]), exp(datur_se[,x])))
    both <- sapply(1:ncol(datur_se), function(x) 
        rpois(length(datur_se[,x]), exp(datur_se[,x] + datur_od[,x])))
    random_covs <- sapply(1:ncol(datur_od), function(x) 
        rpois(length(datur_od[,x]), exp(mean_[,x] + datur_od[,x])))
    spatial_covs <- sapply(1:ncol(datur_se), function(x) 
        rpois(length(datur_se[,x]), exp(mean_[,x] + datur_se[,x])))
    both_covs <- sapply(1:ncol(datur_se), function(x) 
        rpois(length(datur_se[,x]), exp(mean_[,x] + datur_se[,x] + datur_od[,x])))
    
    sp@data[,paste0("over_dispersion", 1:100)] <- datur_od
    sp@data[,paste0("spatial_random", 1:100)] <- datur_se
    sp@data[,paste0("x1", 1:100)] <- x1
    sp@data[,paste0("x2", 1:100)] <- x2
    sp@data[,paste0("random", 1:100)] <- random
    sp@data[,paste0("spatial", 1:100)] <- spatial
    sp@data[,paste0("both", 1:100)] <- both
    sp@data[,paste0("random_covs", 1:100)] <- random_covs
    sp@data[,paste0("spatial_covs", 1:100)] <- spatial_covs
    sp@data[,paste0("both_covs", 1:100)] <- both_covs

    sp
}
                                               
state_data <- apply2(delta_simulatar,"SpatialPolygons", state_data)

# Effects of misspecifying models

Here we run some regressions checking out the affect of not accounting for spatial 
variation as well as not including all variables  

true values are 
1. intecept = 0
2. beta_x1100 = 2
3. beta_x2100 = -3

and the true functional form looks like

$ Y_i \sim Pois(\theta_i)$

$ \theta_i = log(beta_x1100 * x1100_i + beta_x2100 * x2100_i + GMRF(neighbors_i))$ 

In [7]:
df <- state_data$Washington$SpatialPolygons@data

summary(glm(spatial_covs100 ~ x1100 + x2100, data=df, family=poisson))


Call:
glm(formula = spatial_covs100 ~ x1100 + x2100, family = poisson, 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3061  -1.0000  -0.3533   0.1277   6.0919  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.45950    0.12548   3.662  0.00025 ***
x1100        1.75156    0.07803  22.447  < 2e-16 ***
x2100       -3.18249    0.08639 -36.837  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 3055.64  on 38  degrees of freedom
Residual deviance:  124.59  on 36  degrees of freedom
AIC: 213.7

Number of Fisher Scoring iterations: 5


In [8]:
summary(inla(spatial_covs100 ~ x1100 + x2100 + 
                 f(ID, model="besag", graph=state_data$Washington$adjmat,
                   param = c(1, 0.68)), data=df, family="poisson"))


Call:
c("inla(formula = spatial_covs100 ~ x1100 + x2100 + f(ID, model = \"besag\", ",  "    graph = state_data$Washington$adjmat, param = c(1, 0.68)), ",  "    family = \"poisson\", data = df)")

Time used:
 Pre-processing    Running inla Post-processing           Total 
         0.4260          0.0582          0.0393          0.5235 

Fixed effects:
               mean     sd 0.025quant 0.5quant 0.975quant    mode kld
(Intercept)  0.2098 0.2713    -0.3745   0.2278     0.6935  0.2629   0
x1100        2.0422 0.3357     1.4331   2.0231     2.7616  1.9877   0
x2100       -3.3153 0.3723    -4.1119  -3.2938    -2.6405 -3.2532   0

Random effects:
Name	  Model
 ID   Besags ICAR model 

Model hyperparameters:
                   mean     sd 0.025quant 0.5quant 0.975quant   mode
Precision for ID 0.6058 0.3302      0.188   0.5344      1.444 0.4109

Expected number of effective parameters(std dev): 18.27(1.20)
Number of equivalent replicates : 2.135 

Marginal log-Likelihood:  -111.71 

In [9]:
summary(inla(spatial_covs100 ~ x1100 + x2100 + 
                 f(ID, model = "iid", param = c(1, 0.014)), 
             data=df, family="poisson"))


Call:
c("inla(formula = spatial_covs100 ~ x1100 + x2100 + f(ID, model = \"iid\", ",  "    param = c(1, 0.014)), family = \"poisson\", data = df)")

Time used:
 Pre-processing    Running inla Post-processing           Total 
         0.3907          0.0520          0.0324          0.4751 

Fixed effects:
               mean     sd 0.025quant 0.5quant 0.975quant    mode kld
(Intercept)  0.1033 0.3044    -0.5453   0.1208     0.6542  0.1551   0
x1100        1.9774 0.2983     1.4273   1.9634     2.6077  1.9369   0
x2100       -3.2939 0.3601    -4.0516  -3.2781    -2.6276 -3.2488   0

Random effects:
Name	  Model
 ID   IID model 

Model hyperparameters:
                  mean     sd 0.025quant 0.5quant 0.975quant  mode
Precision for ID 1.669 0.8311     0.5904    1.495      3.766 1.199

Expected number of effective parameters(std dev): 18.52(1.253)
Number of equivalent replicates : 2.106 

Marginal log-Likelihood:  -93.65 

In [13]:
summary(glm(spatial_covs100 ~ x2100, data=df, family=poisson))


Call:
glm(formula = spatial_covs100 ~ x2100, family = poisson, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-9.7347  -2.6776  -1.3912  -0.7076  12.9165  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  1.67244    0.07502   22.29   <2e-16 ***
x2100       -2.06174    0.04910  -41.99   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 3055.6  on 38  degrees of freedom
Residual deviance:  768.5  on 37  degrees of freedom
AIC: 855.62

Number of Fisher Scoring iterations: 6


In [14]:
summary(inla(spatial_covs100 ~ x2100 + 
                 f(ID, model="besag", graph=state_data$Washington$adjmat,
                   param = c(1, 0.68)), data=df, family="poisson"))


Call:
c("inla(formula = spatial_covs100 ~ x2100 + f(ID, model = \"besag\", ",  "    graph = state_data$Washington$adjmat, param = c(1, 0.68)), ",  "    family = \"poisson\", data = df)")

Time used:
 Pre-processing    Running inla Post-processing           Total 
         0.4695          0.0728          0.0257          0.5679 

Fixed effects:
               mean     sd 0.025quant 0.5quant 0.975quant    mode kld
(Intercept)  0.2037 0.3033    -0.4612   0.2319     0.7195  0.2976   0
x2100       -2.8520 0.5342    -4.0013  -2.8189    -1.8896 -2.7566   0

Random effects:
Name	  Model
 ID   Besags ICAR model 

Model hyperparameters:
                   mean     sd 0.025quant 0.5quant 0.975quant   mode
Precision for ID 0.0988 0.0432     0.0385    0.091     0.2044 0.0768

Expected number of effective parameters(std dev): 26.10(0.8192)
Number of equivalent replicates : 1.494 

Marginal log-Likelihood:  -124.38 

In [15]:
summary(inla(spatial_covs100 ~ x2100 + 
                 f(ID, model = "iid", param = c(1, 0.014)), 
             data=df, family="poisson"))


Call:
c("inla(formula = spatial_covs100 ~ x2100 + f(ID, model = \"iid\", ",  "    param = c(1, 0.014)), family = \"poisson\", data = df)")

Time used:
 Pre-processing    Running inla Post-processing           Total 
         0.3899          0.0483          0.0229          0.4611 

Fixed effects:
               mean     sd 0.025quant 0.5quant 0.975quant    mode kld
(Intercept)  0.1758 0.4143    -0.7162   0.2036     0.9159  0.2602   0
x2100       -2.6691 0.4976    -3.7282  -2.6426    -1.7614 -2.5924   0

Random effects:
Name	  Model
 ID   IID model 

Model hyperparameters:
                   mean     sd 0.025quant 0.5quant 0.975quant   mode
Precision for ID 0.3138 0.1197     0.1367   0.2954     0.5999 0.2605

Expected number of effective parameters(std dev): 26.97(0.7227)
Number of equivalent replicates : 1.446 

Marginal log-Likelihood:  -108.23 